In [106]:
# TODO
# add latest jersey numbers
# list jersey numbers per season
# add career stats
# add year by year stats that are availble
# season leaders

In [1]:
import re
import pandas as pd
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:
        row=i.split('\t')
        if(len(row[0])>7 and len(row)==1 and ('COACH' not in i and 'OVERALL' not in i and 'ECAC:' not in i and 'CAPTAIN' not in i and 'HOCKEY' not in i and 'NEIHL:' not in i and 'forfeit' not in i)):
            tourneys.append(row[0])
            
            
tourneyDict={}
for i in tourneys:
    if(i=='Key to Tournaments'):
        continue
    tourney=i.split(' ')
    tourneyDict[tourney[0]]=' '.join(tourney[1:])
tourneyDict['nc']='Non-Colleigate'
tourneyDict['Oly nc']='1932 NEAAU Olympic tryouts-Non-Colleigate'
tourneyDict['ex']='Exhibition'
tourneyDict['HF ex']='Hall of Fame Game-Exhibition'
tourneyDict['IB ex']='Ice Breaker-Exhibition'

In [2]:
import re
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:      
        if(len(i)==7):
            season=i
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN.?: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w*) (?:\((.?ot)\))? ?(.*)\t(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(5),
                 'gameType':gameType,
                 'tourney': game.group(6),
                 'scoreline':game.group(7),
                 'location':'',
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['tourney']=='(ex)' or gameDict['result']=='E'):
            gameDict['gameType']='Exhibition'
            gameDict['tourney'] = None
        if(gameDict['arena']=='Agganis Arena' or gameDict['arena']=='Brown Arena' or gameDict['arena']=='Boston Arena'):
            gameDict['location']='Home'
        elif(gameDict['tourney']==None or gameDict['tourney']=='(nc)' or gameDict['tourney'] == '(B1G/HE)' or ((gameDict['tourney'] == '(HE)' or gameDict['tourney'] == '(ECAC)') and (gameDict['arena'] != 'TD Garden' and gameDict['arena'] != 'Boston Garden' and gameDict['arena'] != 'Providence CC'))):
            gameDict['location']='Away'
        if(gameDict['location']=='' or gameDict['arena']=='Boston Garden' or gameDict['arena']=='VW Arena'):
            gameDict['location']='Neutral'
        if((gameDict['arena']=='Gutterson' and gameDict['opponent']=='Vermont') or (gameDict['arena']=='Houston' and gameDict['opponent']=='Rensselaer') or (gameDict['arena']=='Broadmoor' and gameDict['opponent']=='Colorado College') or (gameDict['arena']=='DEC Center' and gameDict['opponent']=='Minnesota Duluth')or (gameDict['arena']=='Magness Arena' and gameDict['opponent']=='Denver')or (gameDict['arena']=='Mariucci Arena' and gameDict['opponent']=='Minnesota')or (gameDict['arena']=='Munn Ice Arena' and gameDict['opponent']=='Michigan State')or (gameDict['arena']=='Walker Arena' and gameDict['opponent']=='Clarkson')or (gameDict['arena']=='Thompson Arena' and gameDict['opponent']=='Dartmouth')or (gameDict['arena']=='St. Louis Arena' and gameDict['opponent']=='St. Louis') or (gameDict['arena']=='Sullivan Arena' and gameDict['opponent']=='Alaska Anchorage')):
            gameDict['location']='Away'

        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        if(int(gameDict['month']) >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=int(gameDict['season'][:4])
        elif(int(gameDict['month']) <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        if(gameDict['season']=='1973-74' and gameDict['date']=='12/12/1973'):
            coach='Jack Parker'
        gameList.append(gameDict)
        teamSet.add(gameDict['opponent'])
f.close()
df=pd.DataFrame(gameList)

In [65]:
teamList=teams.split('|')

In [64]:
teams="Air Force|Alaska Anchorage|American International|Arizona State|Army|Bates|Bentley|Boston College|Bowdoin|Bowling Green|Brown|Clarkson|Colby|Colgate|Colorado College|Connecticut|Cornell|Dalhousie|Dartmouth|Denver|Ferris State|Hamilton College|Harvard|Holy Cross|Lake Superior State|Loyola of Montreal|Maine|Massachusetts|Massachusetts Devens|UMass Lowell|Memorial University|Merrimack|Miami|Michigan|Michigan State|Michigan Tech|Middlebury|Minnesota|Minnesota Duluth|Minnesota State|MIT|Nebraska-Omaha|New Brunswick|New Hampshire|Niagara|North Dakota|Northeastern|Northern Arizona|Northern Michigan|Norwich|Notre Dame|Ohio State|Ohio University|Pennsylvania|Princeton|Providence|Queen’s|Quinnipiac|Rensselaer|Robert Morris|Royal Military College|Sacred Heart|St. Cloud State|St. Francis Xavier|St. Lawrence|St. Louis|Toronto|Tufts|Union|US International|Vermont|Western Michigan|Williams|Wisconsin|Yale|Intercollegiate Totals|Non-Collegiate|Boston Athletic Assoc.|Boston Hockey Club|Brae Burn|Briarcliff Lodge|Eastern Nationals|Jr. Olympics|L’Assoc. St. Dominique|Maple Athletic Assoc.|Needham Hockey Club|New Haven Club|Nichols Club|St. Nicholas|Tech Beavers|Union Boat Club|University Club|U.S. National Team"

In [4]:
print('["',end='')
for i in range(2002,2013):
    print(i,str(i+1)[2:],sep='-',end='","')
    
print('\b\b]')

["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","]


In [22]:
for s in ['2018-19','2019-20','2020-21']:
#for s in ["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13"]:

    print(s)

    resSum={'W':0,'L':0,'T':0}
    for i in ['Home','Away','Neutral']:
        print(i,end=":")
        for d in ['W','L','T']:
            res=df.loc[(df['location']==i) & (df['season']==s) & (df['result']==d) & (df['tourney']!='Exhibition')]['date'].count()
            print(res,end='-')
            resSum[d]+=res
        print("\b")
    print("Total: {}-{}-{}".format(resSum['W'],resSum['L'],resSum['T']))

2018-19
Home:8-7-0-
Away:7-7-4-
Neutral:1-4-0-
Total: 16-18-4
2019-20
Home:8-6-3-
Away:5-6-3-
Neutral:0-1-2-
Total: 13-13-8
2020-21
Home:4-3-0-
Away:6-1-1-
Neutral:0-1-0-
Total: 10-5-1


In [77]:
#df.loc[(df['season']=='2012-13') & (df['location']!='Home')]

In [107]:
x=df.loc[(df['location']!="Home")].groupby(['arena','opponent','location']).size()

In [46]:
df['year']

0       1918
1       1920
2       1920
3       1922
4       1922
        ... 
2793    2022
2794    2022
2795    2022
2796    2022
2797    2022
Name: year, Length: 2798, dtype: int64

In [59]:
df.loc[(df['coach']=="Albie O'Connell")].sort_values('GD',ascending=False)

,date,result,ot,arena,opponent,gameType,tourney,scoreline,location,coach,captain,conference,season,note,month,day,year,BUScore,OppoScore,GD
2796,3/5/2022,L,None,Alfond Arena,Maine,Conference,None,1-8,Away,Albie O'Connell,Logan Cockerill,Hockey East,2021-22,,3,5,2022,1,8,7
2669,10/6/2018,W,None,Agganis Arena,Acadia,Exhibition,None,7-1,Home,Albie O'Connell,"Bobo Carpenter, Dante Fabbro",Hockey East,2018-19,,10,6,2018,7,1,6
2782,1/7/2022,W,None,Agganis Arena,Arizona State,Non-Conference,None,7-1,Home,Albie O'Connell,Logan Cockerill,Hockey East,2021-22,,1,7,2022,7,1,6
2704,3/9/2019,L,None,Alfond Arena,Maine,Conference,None,0-6,Away,Albie O'Connell,"Bobo Carpenter, Dante Fabbro",Hockey East,2018-19,,3,9,2019,0,6,6
2674,10/26/2018,L,None,Agganis Arena,Providence,Conference,None,0-5,Home,Albie O'Connell,"Bobo Carpenter, Dante Fabbro",Hockey East,2018-19,,10,26,2018,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2772,11/12/2021,T,ot,Agganis Arena,Massachusetts,Conference,None,2-2,Home,Albie O'Connell,Logan Cockerill,Hockey East,2021-22,,11,12,2021,2,2,0
2710,10/18/2019,T,ot,Agganis Arena,Northern Michigan,Non-Conference,None,4-4,Home,Albie O'Connell,Patrick Curry,Hockey East,2019-20,,10,18,2019,4,4,0
2758,3/6/2021,T,ot,Tsongas Center,UMass Lowell,Conference,None,3-3,Away,Albie O'Connell,Logan Cockerill,Hockey East,2020-21,,3,6,2021,3,3,0
2702,2/28/2019,T,ot,Schneider Arena,Providence,Conference,None,1-1,Away,Albie O'Connell,"Bobo Carpenter, Dante Fabbro",Hockey East,2018-19,,2,28,2019,1,1,0


In [54]:
(df.loc[(df['season']=="2004-05") & (df['location']!="Home")])

,date,result,ot,arena,opponent,gameType,tourney,scoreline,location,coach,captain,conference,season,note,month,day,year,BUScore,OppoScore,GD
2093,10/8/2004,L,None,Nutter Center,Miami,Non-Conference,Lefty McFadden Invitational,1-5,Neutral,Jack Parker,Brian McConnell,Hockey East,2004-05,,10,8,2004,1,5,4
2094,10/9/2004,L,None,Nutter Center,Michigan,Non-Conference,Lefty McFadden Invitational,2-7,Neutral,Jack Parker,Brian McConnell,Hockey East,2004-05,,10,9,2004,2,7,5
2099,11/5/2004,L,None,VW Arena,New Hampshire,Conference,None,3-4,Neutral,Jack Parker,Brian McConnell,Hockey East,2004-05,,11,5,2004,3,4,1
2101,11/13/2004,W,None,Lawler Rink,Merrimack,Conference,None,3-1,Away,Jack Parker,Brian McConnell,Hockey East,2004-05,,11,13,2004,3,1,2
2102,11/19/2004,W,None,Tsongas Center,UMass Lowell,Conference,None,7-2,Away,Jack Parker,Brian McConnell,Hockey East,2004-05,,11,19,2004,7,2,5
2105,11/26/2004,L,None,Magness Arena,Denver,Non-Conference,None,1-4,Away,Jack Parker,Brian McConnell,Hockey East,2004-05,,11,26,2004,1,4,3
2106,11/27/2004,L,None,World Arena,Colorado College,Non-Conference,None,3-7,Away,Jack Parker,Brian McConnell,Hockey East,2004-05,,11,27,2004,3,7,4
2108,12/4/2004,W,None,Mullins Center,Massachusetts,Conference,None,7-1,Away,Jack Parker,Brian McConnell,Hockey East,2004-05,,12,4,2004,7,1,6
2109,12/11/2004,W,None,Houston,Rensselaer,Non-Conference,None,4-1,Away,Jack Parker,Brian McConnell,Hockey East,2004-05,,12,11,2004,4,1,3
2113,1/7/2005,W,None,Matthews Arena,Northeastern,Conference,None,3-1,Away,Jack Parker,Brian McConnell,Hockey East,2004-05,,1,7,2005,3,1,2


In [105]:
x=df.groupby(['arena','opponent','location']).size()

In [23]:
x=df.loc[(df['location']!="Home")].groupby(['arena','opponent','location']).size()
rows=(x.to_string().split('\n'))
for i in rows:
    print(repr(i))

'arena                     opponent                location'
'Alfond (Colby)            Colby                   Away         8'
'Alfond Arena              Maine                   Away        61'
'Appleton                  Bowling Green           Neutral      1'
'                          Clarkson                Away         1'
'                          St. Lawrence            Away        25'
'BGSU Arena                Bowling Green           Away         2'
'Baker Rink                Princeton               Away        35'
'Bates Rink                Bates                   Away         1'
'Bear Mountain             Army                    Away         1'
'Beaverbrook               Memorial Univ.          Neutral      1'
'                          New Brunswick           Away         2'
'                                                  Neutral      2'
'                          St. Francis Xavier      Neutral      1'
'Beebe Lake                Cornell                 Away         2'
'

In [6]:
df.loc[(df['coach']=="David Quinn")  & (df['season']=='2013-14')  & (df['result']=='W')  &  (df['tourney']!='Exhibition')].count()

date          10
result        10
ot             0
arena         10
opponent      10
gameType      10
tourney        2
scoreline     10
location      10
coach         10
captain       10
conference    10
season        10
note          10
month         10
day           10
year          10
BUScore       10
OppoScore     10
GD            10
dtype: int64

In [28]:
locs=set()
for i in gameList:
    locs.add(i['arena'])

In [189]:
fileName=('JerseyNumbers.txt')
tourneys=[]
teamSet=set()
playerList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    numDict={'number':0,'player':'','season':''}
    for i in rows:
        
        numSearch=re.search("#(.*)",i)
        if numSearch != None:
            number=numSearch.group(1)
        if("Retired - " in i):
                continue
        playerSearch=re.search("(\d*-\d*) (.*)",i)
        if playerSearch != None:
            season=playerSearch.group(1)
            numDict={'number':int(number),
                 'season':convertSeasons(season),
                 'player':playerSearch.group(2)}
            playerList.append(numDict)
f.close()

In [190]:
jerseydf=pd.DataFrame(playerList)

In [191]:
jerseydf

,number,season,player
0,1,1927-28,Sydney Silberberg
1,1,1935-36,Dexter Wesson
2,1,1936-37,Russell Lynch
3,1,1939-40,Henry Sullivan
4,1,"1947-48,1948-49,1949-50",Ralph Bevins
...,...,...,...
861,37,"2021-22,2022-23",Matt Brown
862,39,"1999-00,2000-01,2001-02,2002-03",Andy Warren
863,40,1950-51,Ed Cahoon
864,83,"1998-99,1999-00,2000-01,2001-02",Jason Tapp


In [83]:
#jerseydf.loc[jerseydf.season.str.contains('1994-95') & (jerseydf['number']==2)]

In [130]:
def convertToInt(val):
        if(val.isdigit()):
            val=int(val)
        else:
            val=None
        return val

In [282]:
def convertToFloat(val):
        try:
            val=float(val)
        except:
            val=float('nan')
        return val

In [227]:
def convertSeasons(season):
        gap=season.split(',')
        years=season[2:].split('-')
        seasonStr=''
        if(len(gap)>1):
            for i in gap:
                seasonStr+=convertSeasons(i)
        else:
            yearDiff=abs(int(years[0])-int(years[1]))
            if(yearDiff>6):
                yearDiff=100-yearDiff
            firstHalf=season[:4]
            seasonStr=''
            for i in range(yearDiff):
                secondHalf=int(firstHalf)+1
                seasonStr+=str(firstHalf)+'-'+str(secondHalf)[2:]+','
                firstHalf=secondHalf
        return seasonStr[:-1]

In [228]:
fileName=('SkaterStats.txt')
tourneys=[]
teamSet=set()
skateList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        skaterSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)\/(\S*)',i)
        if skaterSearch!=None:
            skaterDict={'last':skaterSearch.group(1),
                       'first':skaterSearch.group(2),
                        'name':skaterSearch.group(2)+' '+skaterSearch.group(1),
                       'seasons':convertSeasons(skaterSearch.group(3)),
                       'gp':convertToInt(skaterSearch.group(4)),
                       'goals':convertToInt(skaterSearch.group(5)),
                       'assists':convertToInt(skaterSearch.group(6)),
                       'pts':convertToInt(skaterSearch.group(7)),
                       'pen':convertToInt(skaterSearch.group(8)),
                       'pim':convertToInt(skaterSearch.group(9))}
            skateList.append(skaterDict)

In [230]:
dfSkate=pd.DataFrame(skateList)
dfSkate.loc[(dfSkate['seasons'].str.contains('pres'))]

,last,first,name,seasons,gp,goals,assists,pts,pen,pim


In [231]:
dfSkate

,last,first,name,seasons,gp,goals,assists,pts,pen,pim
0,Abbott,Darrell,Darrell Abbott,"1966-67,1967-68,1968-69",92.0,14,50,67,NaN,34.0
1,Abbott,Larry,Larry Abbott,"1970-71,1971-72,1972-73",62.0,4,10,14,5.0,17.0
2,Ahola,Peter,Peter Ahola,"1989-90,1990-91",82.0,15,44,59,68.0,153.0
3,Aiken,John,John Aiken,1954-55,NaN,0,0,0,NaN,4.0
4,Alger,Rich,Rich Alger,"1995-96,1996-97",35.0,1,5,6,15.0,41.0
...,...,...,...,...,...,...,...,...,...,...
757,Yetten,Peter,Peter Yetten,"1968-69,1969-70,1970-71",49.0,0,11,11,NaN,18.0
758,Yip,Brandon,Brandon Yip,"2005-06,2006-07,2007-08,2008-09",139.0,45,63,108,99.0,234.0
759,Young,Scott,Scott Young,"1985-86,1986-87",71.0,31,34,65,NaN,18.0
760,Zancanaro,Brad,Brad Zancanaro,"2002-03,2003-04,2004-05,2005-06",161.0,34,56,90,67.0,153.0


In [264]:
x=jerseydf.loc[jerseydf.season.str.contains('2019-20')]['player']

In [265]:
jerseydf.loc[jerseydf.season.str.contains('2019-20')]

,number,season,player
35,1,"2017-18,2018-19,2019-20",Nico Lynch
74,2,"2019-20,2020-21,2021-22,2022-23",Case McCarthy
105,3,"2017-18,2018-19,2019-20,2020-21",Ty Amonte
139,4,"2018-19,2019-20,2020-21",David Farrance
167,5,"2017-18,2018-19,2019-20",Cam Crotty
225,7,"2019-20,2020-21,2021-22",Alex Vlasic
288,9,"2017-18,2018-19,2019-20,2020-21,2021-22",Logan Cockerill
322,10,"2016-17,2017-18,2018-19,2019-20",Gabriel Chabot
354,11,"2016-17,2017-18,2018-19,2019-20",Patrick Curry
391,12,"2019-20,2020-21,2021-22,2022-23",Jamie Armstrong


In [337]:
for i in x:
    if(dfSkate.loc[dfSkate['name']==i]['name'].count()==0 and dfGoalie.loc[dfGoalie['name']==i]['name'].count()==0):
        print(i)


Case McCarthy
Alex Vlasic
Jamie Armstrong
Trevor Zegras
Sean Driscoll
Markus Boguslavsky
Robert Mastrosimone
Alex Brink
Domenick Fensore
Sam Stevens
John Copeland
Ethan Philips
Ashton Abel
Sam Tucker
Wilmer Skoog


In [257]:
'(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S)'

'(.*),(.*) (\\S*) (\\S*) (\\S*) (\\S*) (\\S*) (\\S*) (\\S*) (\\S*) (\\S*) (\\S)'

In [336]:
fileName=('GoalieStats.txt')
tourneys=[]
teamSet=set()
goalieList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        goalieSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)',i)
        if goalieSearch!=None:
            mins=goalieSearch.group(5).split(':')
            if(len(mins)>1):
                time = "{}:{}".format(*divmod(int(mins[0]), 60)) + ":" + mins[1]
                time = pd.to_timedelta(time)
            else:
                time=float('nan')
            goalieDict={'last':goalieSearch.group(1),
                       'first':goalieSearch.group(2),
                       'name':goalieSearch.group(2)+' '+goalieSearch.group(1),
                       'seasons':convertSeasons(goalieSearch.group(3)),
                       'gp':convertToInt(goalieSearch.group(4)),
                       'mins':round(pd.Timedelta(time).total_seconds()/60,2),
                       'ga':convertToInt(goalieSearch.group(6)),
                       'gaa':convertToFloat(goalieSearch.group(7)),
                       'saves':convertToInt(goalieSearch.group(8)),
                       'sv%':convertToFloat(goalieSearch.group(9)),
                       'W':convertToInt(goalieSearch.group(10)),
                       'L':convertToInt(goalieSearch.group(11)),
                       'T':convertToInt(goalieSearch.group(12))}
            goalieList.append(goalieDict)

In [334]:
dfGoalie=pd.DataFrame(goalieList)

In [335]:
dfGoalie['mins']

0     2291.55
1     1840.20
2       18.20
3     2659.78
4     2770.00
5       40.00
6       80.00
7     5679.93
8     3846.20
9      100.00
10    6271.18
11    4869.63
12    2316.60
13    1611.00
14    1791.22
15    3698.00
16    2550.00
17     670.00
18    4530.50
19      62.62
20    7308.22
21     180.00
22    3823.55
23     966.23
24    4025.77
25        NaN
26     145.27
27    1648.30
28     584.40
29    5643.15
30        NaN
31       7.20
32    3469.87
33    3281.40
34    2401.17
35    8188.32
36      10.35
37    4491.83
38    4422.70
39    6566.63
40       5.00
41     335.00
42     171.55
43     225.45
44    2490.00
45     265.00
46    1570.68
47    2376.60
48        NaN
49     996.20
50    4260.40
51    1360.00
52     620.00
53    2650.07
54    2480.00
55    3318.00
56      11.47
57    2337.50
58       2.05
59      15.00
Name: mins, dtype: float64